$\operatorname{ALGORITHM} \operatorname{GAUSS}\left(\widetilde{\mathbf{A}}=\left[a_{j k}\right]=\left[\begin{array}{ll}\mathbf{A} & \mathbf{b}\end{array}\right]\right)$
This algorithm computes a unique solution $\mathbf{x}=\left[x_j\right]$ of the system (1) or indicates that (1) has no unique solution.

INPUT: Augmented $n \times(n+1)$ matrix $\widetilde{\mathbf{A}}=\left[a_{j k}\right]$, where $a_{j, n+1}=b_j$
OUTPUT: Solution $\mathbf{x}=\left[x_j\right]$ of (1) or message that the system (1) has no unique solution
For $k=1, \cdots, n-1$, do:

1
2
3
4
5
6
7
$$
m=k
$$

For $j=k+1, \cdots, n$, do:
If $\left(\left|a_{m k}\right|<\left|a_{j k}\right|\right)$ then $m=j$
End
If $a_{m k}=0$ then OUTPUT "No unique solution exists"
Stop
[Procedure completed unsuccessfully]
Else exchange row $k$ and row $m$
If $a_{n n}=0$ then OUTPUT "No unique solution exists."
Stop
Else
For $j=k+1, \cdots, n$, do:
$$
m_{j k}:=\frac{a_{j k}}{a_{k k}}
$$

For $p=k+1, \cdots, n+1$, do:
$$
a_{j p}:=a_{j p}-m_{j k} a_{k p}
$$

End
End
End
$$
x_n=\frac{a_{n, n+1}}{a_{n n}} \quad[\text { Start back substitution }]
$$

For $i=n-1, \cdots, 1$, do:
$$
x_i=\frac{1}{a_{i i}}\left(a_{i, n+1}-\sum_{j=i+1}^n a_{i j} x_j\right)
$$

End
OUTPUT $\mathbf{x}=\left[x_j\right]$. Stop
End GAUSS

In [1]:
def gauss_elimination(A):
    """
    Solves a system of linear equations Ax = b using Gaussian elimination with partial pivoting.
    
    Args:
        A: Augmented matrix [A|b] where A is an n x n matrix and b is the n x 1 vector.
        
    Returns:
        x: Solution vector if a unique solution exists.
        or 
        str: "No unique solution exists" if the system has no unique solution.
    
    Reference: Advanced Engineering Mathematics 10th Edition by Erwin Kreyszig (page 849)
    """
    n = len(A)
    
    # Forward elimination with partial pivoting
    for k in range(n - 1):
        # Partial pivoting
        max_row = k
        for j in range(k + 1, n):
            if abs(A[j][k]) > abs(A[max_row][k]):
                max_row = j
        
        # Check for singular matrix
        if A[max_row][k] == 0:
            return "No unique solution exists"
        
        # Swap rows if necessary
        if max_row != k:
            A[k], A[max_row] = A[max_row], A[k]
        
        # Elimination
        for j in range(k + 1, n):
            mjk = A[j][k] / A[k][k]
            for p in range(k, n + 1):
                A[j][p] -= mjk * A[k][p]
    
    # Check for singular matrix
    if A[n-1][n-1] == 0:
        return "No unique solution exists"
    
    # Back substitution
    x = [0] * n
    x[n-1] = A[n-1][n] / A[n-1][n-1]
    
    for i in range(n-2, -1, -1):
        sum_ax = 0
        for j in range(i + 1, n):
            sum_ax += A[i][j] * x[j]
        x[i] = (A[i][n] - sum_ax) / A[i][i]
    
    return x

In [2]:
# Example system:
# 2x + y - z = 8
# -3x - y + 2z = -11
# -2x + y + 2z = -3
# Solution: x = 2, y = 3, z = -1

augmented_matrix = [
    [2, 1, -1, 8],
    [-3, -1, 2, -11],
    [-2, 1, 2, -3]
]

solution = gauss_elimination(augmented_matrix)
print("Solution:", solution)

Solution: [2.0, 3.0000000000000004, -0.9999999999999999]


In [3]:
def gauss_seidel(A, b, x0, epsilon, N):
    """
    Solves a system of linear equations Ax = b using Gauss-Seidel iteration.
    
    Args:
        A: Coefficient matrix (n x n)
        b: Right-hand side vector (n x 1)
        x0: Initial approximation vector (n x 1)
        epsilon: Tolerance for convergence
        N: Maximum number of iterations
        
    Returns:
        x: Approximate solution if converged within tolerance
        or 
        str: Failure message if not converged after N iterations

    Reference: Advanced Engineering Mathematics 10th Edition by Erwin Kreyszig (page 860)
    """
    # ========== DIMENSION CHECKS ==========
    n = len(A)
    
    # Check if A is square
    if any(len(row) != n for row in A):
        raise ValueError("Matrix A must be square (n x n)")
    
    # Check b dimensions
    if len(b) != n:
        raise ValueError(f"Dimension mismatch: b has {len(b)} elements but should have {n} (must match A's rows)")
    
    # Check x0 dimensions
    if len(x0) != n:
        raise ValueError(f"Dimension mismatch: x0 has {len(x0)} elements but should have {n} (must match A's dimension)")
    
    # Check diagonal elements (Gauss-Seidel requirement)
    for i in range(n):
        if A[i][i] == 0:
            raise ValueError(f"Zero diagonal element at A[{i}][{i}] - Gauss-Seidel requires non-zero diagonal elements")
    
    # ========== MAIN ALGORITHM ==========
    x_prev = x0.copy()
    x_next = x0.copy()
    
    for m in range(N):
        # Update each component of the solution vector
        for j in range(n):
            sum1 = 0.0
            for k in range(j):
                sum1 += A[j][k] * x_next[k]
            
            sum2 = 0.0
            for k in range(j+1, n):
                sum2 += A[j][k] * x_prev[k]
            
            x_next[j] = (b[j] - sum1 - sum2) / A[j][j]
        
        # Check convergence condition
        converged = True
        for j in range(n):
            if abs(x_next[j] - x_prev[j]) >= epsilon * abs(x_next[j]):
                converged = False
                break
        
        if converged:
            return x_next
        
        # Prepare for next iteration
        x_prev = x_next.copy()
    
    return "No solution satisfying the tolerance condition obtained after N iteration steps."


In [4]:
# Example system:
# 4x + y + 2z = 4
# x + 5y + 2z = 3
# x + y + 3z = 3
# Solution: x = 0.5, y = 0.5, z = 0.666...

A = [
    [4, 1, 2],
    [1, 5, 2],
    [1, 1, 3]
]
b = [4, 3, 3]

x0 = [0, 0, 0]  # Initial guess
epsilon = 1e-6   # Tolerance
N = 100          # Max iterations

solution = gauss_seidel(A, b, x0, epsilon, N)
print("Solution:", solution)

Solution: [0.5813953627874815, 0.1860464999192942, 0.7441860457644082]


In [5]:
A_gauss =   [
                [-4.0,  1.0,  1.0,  0.0, -200.0],
                [ 1.0, -4.0,  0.0,  1.0, -200.0],
                [ 1.0,  0.0, -4.0,  1.0, -100.0],
                [ 0.0,  1.0,  1.0, -4.0, -100.0],
            ]
uG_vec = gauss_elimination(A_gauss)
print("Solution:", uG_vec)

Solution: [87.5, 87.5, 62.49999999999999, 62.5]


In [6]:
A_GS =  [
        [-4.0,  1.0,  1.0,  0.0],
        [ 1.0, -4.0,  0.0,  1.0],
        [ 1.0,  0.0, -4.0,  1.0],
        [ 0.0,  1.0,  1.0, -4.0]
        ]
b_GS = [-200.0, -200.0, -100.0, -100.0]

x0_GS = [1.0, 1.0, 1.0, 1.0]    # Initial guess
eps = 1e-16             # Tolerance
max_iter = 1000         # Max iterations

uGS_vec = gauss_seidel(A_GS, b_GS, x0_GS, eps, max_iter)
print("Solution:", uGS_vec)

Solution: [87.5, 87.5, 62.5, 62.5]


In [7]:
import numpy as np

In [8]:
n = int(4)
A = np.zeros((n, n))
P = np.zeros((n, n))
b = np.zeros(n)
u = np.zeros(n)

$$
u_{i+1, j} + u_{i, j+1} + u_{i-1, j} + u_{i, j-1} - 4 u_{i, j} = 0
$$

$$
\begin{bmatrix}
u_{03} & u_{13} & u_{23} & u_{33} \\
u_{02} & u_{12} & u_{22} & u_{32} \\
u_{01} & u_{11} & u_{21} & u_{31} \\
u_{00} & u_{10} & u_{20} & u_{30}
\end{bmatrix}
$$

In [9]:
# Initial conditions
P[0, :] = 0.0   # Top side
P[:, 0] = 100.0 # Left side
P[n-1, :] = 100.0 # Right side
P[:, n-1] = 100.0 # Bottom side

P

array([[100.,   0.,   0., 100.],
       [100.,   0.,   0., 100.],
       [100.,   0.,   0., 100.],
       [100., 100., 100., 100.]])

In [10]:
b[0] = -(P[1,3] + P[2,0]) # u_{01} & u_{10}
b

array([-200.,    0.,    0.,    0.])

In [11]:
b[1] = -(P[2,3] + P[3,2]) # u_{31} & u_{20}
b

array([-200., -200.,    0.,    0.])

In [12]:
b[2] = -(P[0,1] + P[1,0]) # u_{13} & u_{02}
b

array([-200., -200., -100.,    0.])

In [13]:
b[3] = -(P[1,3] + P[0,2]) # u_{32} & u_{23}
b

array([-200., -200., -100., -100.])

In [27]:
A = np.array([
    [-4.0,  1.0,  1.0,  0.0],
    [ 1.0, -4.0,  0.0,  1.0],
    [ 1.0,  0.0, -4.0,  1.0],
    [ 0.0,  1.0,  1.0, -4.0]
])
A

array([[-4.,  1.,  1.,  0.],
       [ 1., -4.,  0.,  1.],
       [ 1.,  0., -4.,  1.],
       [ 0.,  1.,  1., -4.]])

In [28]:
x0_GS = [1.0, 1.0, 1.0, 1.0]    # Initial guess
eps = 1e-16             # Tolerance
max_iter = 1000         # Max iterations

u = gauss_seidel(A, b, x0_GS, eps, max_iter)
print("Solution:", u)

Solution: [87.5, 87.5, 62.5, 62.5]


In [16]:
def construct_system(grid):
    """
    Constructs matrix A and vector b for the Laplace equation system Au = b.
    
    Args:
        grid: 2D list representing the temperature grid with boundary conditions
        
    Returns:
        A: Coefficient matrix (4x4 for 4x4 grid with 4 unknowns)
        b: Right-hand side vector
        u: List of (i,j) positions of unknown variables
    """
    n = len(grid)
    u = []
    
    # Find all unknown positions (where value is None)
    for i in range(1, n-1):
        for j in range(1, n-1):
            if grid[i][j] is None:
                u.append((i, j))
    
    num_unknowns = len(u)
    A = [[0.0 for _ in range(num_unknowns)] for _ in range(num_unknowns)]
    b = [0.0 for _ in range(num_unknowns)]
    
    # Create mapping from grid positions to equation indices
    pos_to_idx = {pos: idx for idx, pos in enumerate(u)}
    
    for eq_idx, (i, j) in enumerate(u):
        # Diagonal element
        A[eq_idx][eq_idx] = -4.0
        
        # Check neighbors and fill matrix A and vector b
        for di, dj in [(1,0), (-1,0), (0,1), (0,-1)]:
            ni, nj = i + di, j + dj
            if grid[ni][nj] is None:
                # It's another unknown - find its index
                neighbor_idx = pos_to_idx[(ni, nj)]
                A[eq_idx][neighbor_idx] = 1.0
            else:
                # It's a known boundary value - add to b
                b[eq_idx] -= grid[ni][nj]
    
    return A, b, u

In [17]:
def gauss2(A, b):
    """
    Solves a system of linear equations Ax = b using Gaussian elimination.
    
    Args:
        A: Coefficient matrix (n x n)
        b: Right-hand side vector (n x 1)
        
    Returns:
        x: Solution vector if a unique solution exists
        or 
        str: "No unique solution exists" if the system has no unique solution
    """
    n = len(A)
    
    # Create augmented matrix
    aug = [row.copy() + [b[i]] for i, row in enumerate(A)]
    
    # Forward elimination with partial pivoting
    for k in range(n - 1):
        # Partial pivoting
        max_row = k
        for j in range(k + 1, n):
            if abs(aug[j][k]) > abs(aug[max_row][k]):
                max_row = j
        
        # Check for singular matrix
        if aug[max_row][k] == 0:
            return "No unique solution exists"
        
        # Swap rows if necessary
        if max_row != k:
            aug[k], aug[max_row] = aug[max_row], aug[k]
        
        # Elimination
        for j in range(k + 1, n):
            mjk = aug[j][k] / aug[k][k]
            for p in range(k, n + 1):
                aug[j][p] -= mjk * aug[k][p]
    
    # Check for singular matrix
    if aug[n-1][n-1] == 0:
        return "No unique solution exists"
    
    # Back substitution
    x = [0] * n
    x[n-1] = aug[n-1][n] / aug[n-1][n-1]
    
    for i in range(n-2, -1, -1):
        sum_ax = 0
        for j in range(i + 1, n):
            sum_ax += aug[i][j] * x[j]
        x[i] = (aug[i][n] - sum_ax) / aug[i][i]
    
    return x

In [18]:
def gauss_seidel2(A, b, x0=None, max_iter=100, tol=1e-6, verbose=False):
    """
    Solves a system of linear equations Ax = b using Gauss-Seidel iteration.
    
    Args:
        A: Coefficient matrix (n x n)
        b: Right-hand side vector (n x 1)
        x0: Initial guess (defaults to zero vector)
        max_iter: Maximum number of iterations
        tol: Tolerance for convergence
        verbose: Whether to print iteration info
        
    Returns:
        x: Approximate solution (always returns the latest approximation)
    """
    n = len(A)
    
    # Initialize solution vector
    x = x0.copy() if x0 else [0.0 for _ in range(n)]
    
    # Check diagonal elements
    for i in range(n):
        if A[i][i] == 0:
            raise ValueError(f"Zero diagonal element at A[{i}][{i}]")
    
    for iteration in range(max_iter):
        x_prev = x.copy()
        
        for i in range(n):
            sigma = 0.0
            for j in range(n):
                if j != i:
                    sigma += A[i][j] * x[j]
            x[i] = (b[i] - sigma) / A[i][i]
        
        # Calculate maximum relative error
        max_error = max(abs(x[i] - x_prev[i]) / abs(x[i]) if x[i] != 0 else 0 
                        for i in range(n))
        
        if verbose:
            print(f"Iteration {iteration+1}: Max error = {max_error:.6f}")
        
        if max_error < tol:
            if verbose:
                print(f"Converged after {iteration+1} iterations")
            break
    
    # Always return the current approximation
    return x

In [19]:
def solve_plate_temp(boundary_conditions, method='gauss', max_iter=100, tol=1e-6, verbose=False):
    """
    Solves the temperature distribution problem for a square plate.
    
    Args:
        boundary_conditions: Dictionary of {(i,j): value} for boundary points
        method: 'gauss' for Gaussian elimination or 'liebmann' for Gauss-Seidel
        max_iter: Maximum iterations for Liebmann's method
        tol: Tolerance for Liebmann's method
        verbose: Whether to print iteration info
        
    Returns:
        solution: Complete temperature grid with solved values
        method_used: String indicating which method was used
    """
    # Create grid with None for unknown points
    n = 4  # For 4x4 grid
    grid = [[None for _ in range(n)] for _ in range(n)]
    
    # Apply boundary conditions
    for (i, j), value in boundary_conditions.items():
        grid[i][j] = value
    
    # Construct system
    A, b, u = construct_system(grid)
    
    # Select and apply solution method
    method_used = ""
    if method.lower() in ['gauss', 'elimination', 'direct']:
        method_used = "Gauss elimination"
        solution = gauss2(A, b)
    elif method.lower() in ['liebmann', 'gauss-seidel', 'iterative']:
        method_used = "Liebmann's method (Gauss-Seidel)"
        solution = gauss_seidel2(A, b, x0=[100]*len(b), max_iter=max_iter, tol=tol, verbose=verbose)
    else:
        raise ValueError("Invalid method. Choose 'gauss' or 'liebmann'")
    
    # Fill in the solved values (regardless of convergence)
    for idx, (i, j) in enumerate(u):
        if isinstance(solution, list) and idx < len(solution):
            grid[i][j] = solution[idx]
        else:
            grid[i][j] = None
    
    return grid, method_used

In [20]:
def solve_plate_temperature(boundary_conditions, method='gauss', max_iter=100, tol=1e-6, verbose=False):
    """
    Solves the temperature distribution problem for a square plate.
    
    Args:
        boundary_conditions: Dictionary of {(i,j): value} for boundary points
        method: 'gauss' for Gaussian elimination or 'liebmann' for Gauss-Seidel
        max_iter: Maximum iterations for Liebmann's method
        tol: Tolerance for Liebmann's method
        verbose: Whether to print iteration info
        
    Returns:
        solution: Complete temperature grid with solved values
        method_used: String indicating which method was used
    """
    # Create grid with None for unknown points
    n = 4  # For 4x4 grid
    grid = [[None for _ in range(n)] for _ in range(n)]
    
    # Apply boundary conditions
    for (i, j), value in boundary_conditions.items():
        grid[i][j] = value
    
    # Construct system
    A, b, u = construct_system(grid)
    
    # Select and apply solution method
    method_used = ""
    if method.lower() in ['gauss', 'elimination', 'direct']:
        method_used = "Gauss elimination"
        solution = gauss2(A, b)
    elif method.lower() in ['liebmann', 'gauss-seidel', 'iterative']:
        method_used = "Liebmann's method (Gauss-Seidel)"
        solution = gauss_seidel2(A, b, x0=[100]*len(b), max_iter=max_iter, tol=tol, verbose=verbose)
    else:
        raise ValueError("Invalid method. Choose 'gauss' or 'liebmann'")
    
    # Handle solution results
    if isinstance(solution, str):
        print(f"{method_used} failed: {solution}")
        return grid, method_used
    
    # Fill in the solved values
    for idx, (i, j) in enumerate(u):
        grid[i][j] = solution[idx]
    
    return grid, method_used

def print_temperature_grid(grid, title="Temperature Distribution"):
    """
    Prints the temperature grid in a visually appealing format.
    
    Args:
        grid: 2D list of temperature values
        title: Header for the printed output
    """
    n = len(grid)
    # Determine the maximum width needed for any element
    max_width = max(len(f"{val:.1f}") if val is not None else len("None") 
                   for row in grid for val in row)
    
    print(f"\n{title}:")
    print("+" + ("-" * (max_width + 2) + "+") * n)
    
    for i, row in enumerate(grid):
        print("|", end="")
        for val in row:
            if val is None:
                print(f" {'None':>{max_width}} |", end="")
            else:
                print(f" {val:>{max_width}.1f} |", end="")  # Fixed this line
        print("\n+" + ("-" * (max_width + 2) + "+") * n)

In [21]:
def print_boundary_conditions(boundary_conditions, grid_size=4):
    """
    Prints the boundary conditions with unknown nodes marked as u_{ij}.
    
    Args:
        boundary_conditions: Dictionary of {(i,j): value} for boundary points
        grid_size: Size of the grid (default 4 for 4x4 grid)
    """
    # Determine the maximum width needed for any element
    max_val_width = max(len(str(val)) for val in boundary_conditions.values())
    max_label_width = max(len(f"u_{i}{j}") for i in range(grid_size) 
                                      for j in range(grid_size))
    max_width = max(max_val_width, max_label_width)
    
    print("\nBoundary Conditions with Unknown Nodes:")
    print("+" + ("-" * (max_width + 2) + "+") * grid_size)
    
    for i in range(grid_size):
        print("|", end="")
        for j in range(grid_size):
            pos = (i, j)
            if pos in boundary_conditions:
                # Known boundary value
                print(f" {boundary_conditions[pos]:>{max_width}} |", end="")
            else:
                # Unknown node - print as u_{ij}
                print(f" {'u_'+str(i)+str(j):>{max_width}} |", end="")
        print("\n+" + ("-" * (max_width + 2) + "+") * grid_size)

In [22]:
# Example boundary conditions (using matrix notation from problem)
# (No idea what should be happening in the corners since they are not considered)
boundary_conditions = {
    (0, 0): 100, (0, 1):   0, (0, 2):   0, (0, 3): 100,   # Left side and top-left
    (1, 0): 100,                           (1, 3): 100,   # Left and top
    (2, 0): 100,                           (2, 3): 100,   # Left and top
    (3, 0): 100, (3, 1): 100, (3, 2): 100, (3, 3): 100    # Bottom side and top-right
}

print_boundary_conditions(boundary_conditions)


Boundary Conditions with Unknown Nodes:
+------+------+------+------+
|  100 |    0 |    0 |  100 |
+------+------+------+------+
|  100 | u_11 | u_12 |  100 |
+------+------+------+------+
|  100 | u_21 | u_22 |  100 |
+------+------+------+------+
|  100 |  100 |  100 |  100 |
+------+------+------+------+


In [23]:
# Solve with Gaussian elimination
solution_gauss, method_gauss = solve_plate_temperature(
    boundary_conditions, method='gauss'
)
print_temperature_grid(solution_gauss, f"Solution using {method_gauss}")


Solution using Gauss elimination:
+-------+-------+-------+-------+
| 100.0 |   0.0 |   0.0 | 100.0 |
+-------+-------+-------+-------+
| 100.0 |  62.5 |  62.5 | 100.0 |
+-------+-------+-------+-------+
| 100.0 |  87.5 |  87.5 | 100.0 |
+-------+-------+-------+-------+
| 100.0 | 100.0 | 100.0 | 100.0 |
+-------+-------+-------+-------+


In [24]:
# Solve with Liebmann's method
max_iter = 5
solution_liebmann, method_liebmann = solve_plate_temp(
    boundary_conditions, method='liebmann', max_iter=max_iter, tol=1e-6, verbose=True
)
print_temperature_grid(solution_liebmann, f"Solution using {method_liebmann}")

Iteration 1: Max error = 0.454545
Iteration 2: Max error = 0.142857
Iteration 3: Max error = 0.037037
Iteration 4: Max error = 0.009346
Iteration 5: Max error = 0.002342

Solution using Liebmann's method (Gauss-Seidel):
+-------+-------+-------+-------+
| 100.0 |   0.0 |   0.0 | 100.0 |
+-------+-------+-------+-------+
| 100.0 |  62.5 |  62.5 | 100.0 |
+-------+-------+-------+-------+
| 100.0 |  87.5 |  87.5 | 100.0 |
+-------+-------+-------+-------+
| 100.0 | 100.0 | 100.0 | 100.0 |
+-------+-------+-------+-------+
